In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy.spatial import distance

In [2]:
def is_inside(o,i):
    ox,oy,ow,oh=o
    ix,iy,iw,ih=i
    return ox>ix and oy>iy and ox+ow< ix+iw and oy+oh<iy+ih

In [3]:
# calculate the distance of two point
def sel_person(pre_id,cur_id):
    dis=distance.euclidean(pre_id,cur_id)
    if dis<200:
        return 1
    else:
        return 0


In [4]:
def compare_core(core_list,x1,y1,h,w):
    #dzb=[(y1,x1),(y1,x1+w),(y1+h,x1),(y1+h,x1+w)]
    flag=0
    for i in core_list:
        if i[1]>=y1 and i[1]<=y1+h and i[0]>=x1 and i[0]<=x1+w:
            flag+=1
    return flag
    

In [5]:
def mouse_detect(event,x,y,flags,param):
    global img,p1,p2,core_list
    img_c=img.copy()
    if event==cv2.EVENT_LBUTTONDOWN:
        p1=(x,y)
        cv2.circle(img_c,p1,5,(255,0,0),2)
        cv2.imshow("img",img_c)
    elif event==cv2.EVENT_MOUSEMOVE and (flags & cv2.EVENT_FLAG_LBUTTON):
        cv2.rectangle(img_c,p1,(x,y),(255,0,0),2)
        cv2.imshow("img",img_c)
    elif event==cv2.EVENT_LBUTTONUP:
        p2=(x,y)
        cv2.rectangle(img_c,p1,p2,(255,0,0),2)
        cv2.imshow("img",img_c)
        x1=min(p1[0],p2[0])
        y1=min(p1[1],p2[1])
        w=abs(p2[0]-p1[0])
        h=abs(p2[1]-p1[1])
        flag=compare_core(core_list,x1,y1,h,w)
        rectangle_img=img[y1:y1+h,x1:x1+w]
        #cv2.namedWindow("rectangle_img")
        fond= cv2.FONT_HERSHEY_SIMPLEX
        if flag==0:
            text=f'There is no people.'
        elif flag==1:
            text=f'There is one people.'
        else:
            text=f'There are {flag} people.'
        cv2.putText(rectangle_img,text,(50,50),fond,1,(255,0,0),2)
        
        cv2.imwrite("rectangel_img.png",rectangle_img)
#This code refers to the code of this URL(https://blog.csdn.net/qq_37674858/article/details/80434618)

In [6]:
def task3_group(l):
    c={}
    for i in range(len(l)):
        c[i]=[]
        for j in range(len(l)):
            c[i]+=[distance.euclidean(l[i],l[j])] # c是一个字典，存着每个点到其他点的距离（包括自己）
    ''''''
    #3.1
    id=[] # 存每一个点是否in group的list
    for i in c:
        f1=0
        for j in c[i]:
            if j>0 and j<=300: # 如果距离小于三百大于零，那么就是in group
                f1=1 # 标为1
        id+=[f1]

    c1=0
    c2=0
    # 计算id这个list里面1有几个0有几个
    for i in id:
        if i==1:
            c1+=1
        else:
            c2+=1
    #print(c1)#in group的点
    #print(c2)#not in group的点
    ''''''
    #3.2
    f2=0
    for i in c:
        for j in c[i]:
            if j>300 and j<600:#距离300-600
                f2=1
    if f2==1:
        s2="there is group formation and group destruction"
    else:
        s2="there is no group formation and group destruction"
    #print(f2) #返回1说明该帧中团队产生或破坏，0则无

    ''''''
    #3.3
    f3=0
    for i in c:
        for j in c[i]:
            if j>=600: # 距离大于600
                f3=1
    if f3==1:
        s3="there is entering or leaving"
    else:
        s3="there is no entering or leaving"
    #print(f3) # 返回1 表示有人离队
    return c1,c2,s2,s3

In [7]:
def draw_person(image,person,color,p):
    x,y,w,h=person
    # x,y is the situation of person
    cv2.rectangle(image,(x,y),(x+w,y+h),color,3) # draw the box of person
    cv2.circle(image,(x+int(w/2),y+int(h/2)),5,color,-1) # draw the center of box
    
    # draw the trajectory for each person
    if p:
        for i in p:
            cv2.circle(image,i,5,p[i],-1)
    

In [8]:
# detect and draw in the image
def detect_p(file,pre_point,point): 
    # file is the route, pre_point is the situation of previous frame, point is a dictionary store are the point
    img=cv2.imread(file)
    hog=cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    found,w=hog.detectMultiScale(img)
    fl=[]
    # fl is a list that store found people
    for i,j in enumerate(found):
        flag=0
        for m,n in enumerate(found):
            if i!=m and is_inside (j,n):
                flag=1
        if flag==0:
            fl.append(j)
    # draw box
    # for the 1st frame
    core_list=[]
    if not pre_point:
        
        for p in fl:
            x,y,w,h=p
            x=x+int(w/2)
            y=y+int(h/2)
            pre_point[(x,y)]=(random.randint(0,256),random.randint(0,256),random.randint(0,256))
            draw_person(img,p,pre_point[(x,y)],point)
            core_list.append((x,y))
    # for the frame after
    else:
        temp_d={}# a dictionary store the person in this image
        for p in fl: # search in found list
            x,y,w,h=p
            x=x+int(w/2)
            y=y+int(h/2)
            core_list.append((x,y))
            l=len(pre_point)
            f=0
            # compare with previous frame
            # if the distance is less than threshold, the two person will be regard as the same person
            # if the distance with all points in previous frame are larger than threshold, the person will be a new one
            for i in pre_point:
                if sel_person(i,(x,y))==0:
                    f+=1
                else:
                    temp_d[(x,y)]=pre_point[i]
                    point[(x,y)]=pre_point[i]
                    draw_person(img,p,temp_d[(x,y)],point)

            if f==l:
                temp_d[(x,y)]=(random.randint(0,256),random.randint(0,256),random.randint(0,256))
                point[(x,y)]=temp_d[(x,y)]
                draw_person(img,p,temp_d[(x,y)],point)
            
        pre_point=temp_d # in the last, change the temporary dictionary as previous point
    return img,fl,point,file,core_list
    #if len(fl)<len(list(set(list(point.values())))):
        #fond= cv2.FONT_HERSHEY_SIMPLEX
        #text=f'There are {len(fl)} people, There have been {len(list(set(list(point.values()))))} people'
        #cv2.putText(img,text,(200,200),fond,2,(255,0,0),3)
        #cv2.imwrite(file[-6:-4]+'.png',img)
    #else:
        #fond= cv2.FONT_HERSHEY_SIMPLEX
        #text=f'There are {len(fl)} people, There have been {len(fl)} people'
        #cv2.putText(img,text,(200,200),fond,2,(255,0,0),3)
        #cv2.imwrite(file[-6:-4]+'.png',img)

In [9]:
pre={}
point={}
for i in range(120):
    s=str(i+1)
    if len(s)==1:
        n='step_images/train/STEP-ICCV21-02/00000'+s+'.jpg'
    elif len(s)==2:
        n='step_images/train/STEP-ICCV21-02/0000'+s+'.jpg'
    else:
        n='step_images/train/STEP-ICCV21-02/000'+s+'.jpg'

    p1=0
    p2=0
    img,fl,point,file,core_list=detect_p(n,pre,point)
    #this part is about task2 2.3 2.4, This part needs to be run separately and cannot run at the same time as B

    cv2.namedWindow("img")
    cv2.setMouseCallback("img",mouse_detect)
    cv2.imshow("img",img)
    cv2.waitKey(0)
    
    #B
    if len(fl)<len(list(set(list(point.values())))):
        c1,c2,s2,s3=task3_group(core_list)
        fond= cv2.FONT_HERSHEY_SIMPLEX
        text1='people: '+str(len(fl))+ ' ,total people: '+str(len(list(set(list(point.values())))))
        text2='in group: '+str(c1)
        text3='not in group: '+str(c2)
        cv2.putText(img,text1,(120,100),fond,1,(255,0,0),2)
        cv2.putText(img,text2,(120,150),fond,1,(255,0,0),2)
        cv2.putText(img,text3,(120,200),fond,1,(255,0,0),2)
        cv2.putText(img,s2,(120,250),fond,1,(255,0,0),2)
        cv2.putText(img,s3,(120,300),fond,1,(255,0,0),2)
        cv2.imwrite(file[-7:-4]+'.png',img)
    else:
        c1,c2,s2,s3=task3_group(core_list)
        fond= cv2.FONT_HERSHEY_SIMPLEX
        text1='people: '+str(len(fl))+ ' ,total people: '+str(len(fl))
        text2='in group: '+str(c1)
        text3='not in group: '+str(c2)
        cv2.putText(img,text1,(120,100),fond,1,(255,0,0),2)
        cv2.putText(img,text2,(120,150),fond,1,(255,0,0),2)
        cv2.putText(img,text3,(120,200),fond,1,(255,0,0),2)
        cv2.putText(img,s2,(120,250),fond,1,(255,0,0),2)
        cv2.putText(img,s3,(120,300),fond,1,(255,0,0),2)
        cv2.imwrite(file[-7:-4]+'.png',img)


    

In [10]:
fps = 30  # 视频每秒24帧
size = (1920, 1080) # 需要转为视频的图片的尺寸
# 可以使用cv2.resize()进行修改

video = cv2.VideoWriter("my.mp4",cv2.VideoWriter_fourcc(*"mp4v"), fps, size)#创建.avi
# 视频保存在当前目录下



for i in range(120):
    if i<10:
        img=cv2.imread('00'+str(i)+'.png')
        #img=cv2.resize(img,(604, 604))
        video.write(img)
    elif 10<=i<100:
        img=cv2.imread('0'+str(i)+'.png')

        video.write(img)
    else:
        img=cv2.imread(str(i)+'.png')
        #img=cv2.resize(img,(604, 604))
        video.write(img)

video.release()
cv2.destroyAllWindows()